In [1]:
!pip install flask-ngrok

In [0]:
from flask_ngrok import run_with_ngrok
from flask import Flask,request,jsonify

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words("english"))
import re
import numpy as np
nltk.download('wordnet')
lema = nltk.wordnet.WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import gensim

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/datasets/GoogleNews-vectors-negative300.bin.gz', binary=True)
vocab_size = 300

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def getVectors(corpus,vectors,vocab_size):
    wordset = set(vectors.wv.index2word) #Checks if the word is in the Word2vec corpus 
    counter = 0    
    featureVec = np.zeros(vocab_size,dtype="object")
    for word in corpus:
      if word in wordset:
        featureVec = np.add(featureVec,vectors[word])
      counter = counter + 1
      #print(counter)
    #featureVec = np.divide(featureVec,counter)
    return featureVec

In [0]:
def preProccessing(sent):
  #sent = re.sub('[^a-zA-Z]',' ',sent)
  sent = [word for word in sent.lower().split(' ') if word not in stopwords]
  return sent

In [0]:
def similarity(docs,document):
    data = []
    sim = cosine_similarity(docs,document)
    sim = sim.flatten()
    sim = np.array(sim)
    #sim = sim.reshape(-1,1)
    #sim = np.sort(sim)
    for i in range(len(sim)):
      if sim[i] >= 1.0:
        continue
      data.append((i,sim[i]))
    sim = sim[~(sim >= 1.0)]
    data.sort(key = lambda x: x[1])
    return data


In [0]:
app = Flask(__name__)
run_with_ngrok(app)

In [0]:
@app.route('/similarity/<uuid>',methods = ['GET','POST'])
def home(uuid):
  docs = []
  content = request.json
  content = content['blogs']
  docs.extend(content)
  proccessed = []
  for doc in docs:
    doc = preProccessing(doc)
    doc = getVectors(corpus = doc,vectors = model,vocab_size = vocab_size)
    proccessed.append(doc)
  docs = []
  i = 0
  for doc in proccessed:
    proccessed = np.array(proccessed)
    doc = doc.reshape(1,-1)
    sim = similarity(proccessed,doc)
    docs.append((i,sim))
    i = i+1
    #print(i)
  print(docs)
  data = {"data" : docs}
  return jsonify(data)


In [62]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7ff078b7.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
127.0.0.1 - - [13/Jan/2020 22:43:37] "POST /similarity/123 HTTP/1.1" 200 -


[(0, [(2, 0.5594000719175809), (3, 0.6729258068998232), (1, 0.9440223567374592)]), (1, [(2, 0.5894161286444459), (3, 0.6772768928598724), (0, 0.9440223567374592)]), (2, [(0, 0.5594000719175809), (3, 0.5662745612248996), (1, 0.5894161286444459)]), (3, [(2, 0.5662745612248996), (0, 0.6729258068998232), (1, 0.6772768928598724)])]
